In [3]:
#init
import torch, numpy, sys
sys.path.append("..")
from chatbot_end2end.module.interact_session import InteractSession
from nlptools.utils import Config
import torch.nn as nn

cfg = Config("../config/elsa.yml")
session = InteractSession.build(cfg)

{'encoder': {'vocab_size': 30522, 'hidden_size': 768, 'num_hidden_layers': 12, 'num_attention_heads': 12, 'hidden_act': 'gelu', 'intermediate_size': 3072, 'hidden_dropout_prob': 0.1, 'attention_probs_dropout_prob': 0.1, 'max_position_embeddings': 512, 'type_vocab_size': 2, 'initializer_range': 0.02, 'layer_norm_eps': 1e-12}, 'decoder': {'num_hidden_layers': 1, 'num_attention_heads': 2, 'intermediate_size': 1024, 'shared_embed': True}}


In [4]:
skill_name = "cornell"
skill = session.topic_manager.skills[skill_name]
state_dict = skill.model.decoder.state_dict()
dialog_status = session.new_dialog()
dialog_status.add_utterance("I would hardly say you look like trash")
data = dialog_status.status2data()
data.to("cuda:0")

encoder_out, encoder_hidden = skill.model.dialog_embedding(data["utterance"].data, data["utterance_mask"].data, data["sentiment"].data)
utterance_mask = data["utterance_mask"].data
print(encoder_out.shape)
print(utterance_mask.shape)

torch.Size([1, 20, 768])
torch.Size([1, 20])


In [9]:
import nlptools.zoo.encoders.transformer
import chatbot_end2end.model.generative_tracker
import importlib
importlib.reload(nlptools.zoo.encoders.transformer)
importlib.reload(chatbot_end2end.model.generative_tracker)

from nlptools.zoo.encoders.transformer import TransformerDecoder
from chatbot_end2end.model.generative_tracker import GenerativeTracker

embedding = skill.model.encoder.embedding

decoder_hidden_layers = skill.model.config["decoder"]["num_hidden_layers"]
decoder_attention_heads = skill.model.config["decoder"]["num_attention_heads"]
decoder_hidden_size = skill.model.config["decoder"]["intermediate_size"]

# bert_model_name = skill.model.config["bert_model_name"]
pad_id = skill.model.pad_id
bos_id = skill.model.bos_id
eos_id = skill.model.eos_id
unk_id = skill.model.unk_id
beam_size = 3
len_penalty = 1
unk_penalty = 1
dropout=0

shared_layers = {"encoder": skill.model.encoder}

tracker = GenerativeTracker(shared_layers=shared_layers, skill_name=skill_name, decoder_hidden_laers=decoder_hidden_layers,
                             decoder_attention_heads=decoder_attention_heads, decider_hidden_size=decoder_hidden_size, dropout=dropout, pad_id=pad_id, bos_id=bos_id,
                            eos_id=eos_id, unk_id=unk_id, beam_size=beam_size, len_penalty=len_penalty, unk_penalty=unk_penalty)
tracker = tracker.to("cuda:0")

decoder = TransformerDecoder(embedding, decoder_hidden_layers, decoder_attention_heads, decoder_hidden_size, dropout)
decoder = decoder.to("cuda:0")
decoder.load_state_dict(state_dict)

tracker.decoder = decoder

tracker.beam_search(encoder_out, utterance_mask)

{'encoder': {'vocab_size': 30522, 'hidden_size': 768, 'num_hidden_layers': 12, 'num_attention_heads': 12, 'hidden_act': 'gelu', 'intermediate_size': 3072, 'hidden_dropout_prob': 0.1, 'attention_probs_dropout_prob': 0.1, 'max_position_embeddings': 512, 'type_vocab_size': 2, 'initializer_range': 0.02, 'layer_norm_eps': 1e-12}, 'decoder': {'num_hidden_layers': 1, 'num_attention_heads': 2, 'intermediate_size': 1024, 'shared_embed': True}}
output_buf torch.Size([3, 1])
output_probs1 tensor([[[-22.1560, -21.8709, -22.0562,  ..., -20.7221, -21.4406, -23.3961]],

        [[-22.1560, -21.8709, -22.0562,  ..., -20.7221, -21.4406, -23.3961]],

        [[-22.1560, -21.8709, -22.0562,  ..., -20.7221, -21.4406, -23.3961]]],
       device='cuda:0', grad_fn=<LogSoftmaxBackward>) torch.Size([3, 1, 30522])
output_probs2 tensor([[-22.1560, -21.8709, -22.0562,  ..., -20.7221, -21.4406, -23.3961],
        [-22.1560, -21.8709, -22.0562,  ..., -20.7221, -21.4406, -23.3961],
        [-22.1560, -21.8709, -22.0

(tensor([[  101, 26152, 26152, 26152, 26152, 26152, 26152, 26152, 26152, 26152,
          26152, 26152, 26152, 26152, 26152, 26152, 26152, 26152, 26152, 26152]],
        device='cuda:0'), array([-3.73828952e-05]))

In [ ]:
tracker